In [1]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import torch.optim as optim
import random, os
from models.alexnet import *
from utils import *
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import ConfigParser
Config = ConfigParser.ConfigParser()
Config.read("config.ini")
path_to_FOURVIERE_CLEAN2 = Config.get("Corpus", 'path_to_FOURVIERE_CLEAN2')

In [3]:
cudnn.benchmark = True #-- uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms.
                       #-- If this is set to false, uses some in-built heuristics that might not always be fastest.

cudnn.fastest = True #-- this is like the :fastest() mode for the Convolution modules,
                     #-- simply picks the fastest convolution algorithm, rather than tuning for workspace size

In [4]:
imSize = 225
batchSize = 32
nb_epoch = 50

In [5]:
pathToImg = path_to_FOURVIERE_CLEAN2+"IMAGE_FROM_V/"

# list imgaes
pathImgTrain = [line.split(' ')[1] for line in open(path_to_FOURVIERE_CLEAN2+"/cutTrainTest/u3_train.txt").read().splitlines()]
pathImgTest = [line.split(' ')[1] for line in open(path_to_FOURVIERE_CLEAN2+"/cutTrainTest/u3_test_requestInTrain.txt").read().splitlines()]

ImgNameToClass = {line.split(' ')[1]:line.split(' ')[0] for line in open(path_to_FOURVIERE_CLEAN2+"/cutTrainTest/u3_train.txt").read().splitlines()}
# add image from test
ImgNameToClass.update({line.split(' ')[1]:line.split(' ')[0] for line in open(path_to_FOURVIERE_CLEAN2+"/cutTrainTest/u3_test_requestInTrain.txt").read().splitlines()})
ClassToIdClass = {Artworks:i for i, Artworks in enumerate(sorted(set([line.split(' ')[0] for line in open(path_to_FOURVIERE_CLEAN2+"/cutTrainTest/u3_train.txt").read().splitlines()])))}
ClassToIdClass.update({Artworks:i for i, Artworks in enumerate(sorted(set([line.split(' ')[0] for line in open(path_to_FOURVIERE_CLEAN2+"/cutTrainTest/u3_test_requestInTrain.txt").read().splitlines()])))})
ImgNameToIdClass = {ImgName:ClassToIdClass[ImgNameToClass[ImgName]] for ImgName in  pathImgTrain}
ImgNameToIdClass.update({ImgName:ClassToIdClass[ImgNameToClass[ImgName]] for ImgName in  pathImgTest})

# load image in memory
dImgTest = {imgName:Image.open(pathToImg+imgName).resize((imSize, imSize), Image.BILINEAR) for imgName in pathImgTest}
dImgTrain = {imgName:Image.open(pathToImg+imgName) for imgName in pathImgTrain}

In [6]:
mean = ComputeMean(dImgTrain.values())
std = ComputeStdDev(dImgTrain.values())

In [7]:
imageTrainTransform = transforms.Compose([
    transforms.Scale(300), 
    transforms.RandomCrop(225), 
    transforms.ToTensor(), 
    transforms.Normalize(mean = mean,
                         std = std),
])

imageTestTransform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize(mean = mean,
                         std = std),
])


In [8]:
criterion = nn.CrossEntropyLoss()

## test fixe features parameters

In [9]:
alexTunedClassifier = alexnet(len(ImgNameToIdClass)).train()
copyFeaturesParametersAlexnet(alexTunedClassifier, models.alexnet(pretrained=True))
alexTunedClassifier = alexTunedClassifier.cuda()

copy Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
copy Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
copy Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
copy Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
copy Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [10]:
#define the optimizer to only the classifier with lr of 1e-3
optimizer=optim.SGD([{'params': alexTunedClassifier.classifier.parameters()},
                     {'params': alexTunedClassifier.features.parameters(), 'lr': 0.0}
                    ], lr=0.01, momentum=0.9)
for epoch in range(50): 
    print "epoch", epoch, 
    trainclassifier(alexTunedClassifier, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClass, 1)
    testClassifier(alexTunedClassifier, dImgTest, imSize, imageTestTransform, ImgNameToIdClass, 1)

epoch 0 test : #Correct 174 on 624 (27.9%)
epoch 1 test : #Correct 231 on 624 (37.0%)
epoch 2 test : #Correct 197 on 624 (31.6%)
epoch 3 test : #Correct 226 on 624 (36.2%)
epoch 4 test : #Correct 238 on 624 (38.1%)
epoch 5 test : #Correct 233 on 624 (37.3%)
epoch 6 test : #Correct 217 on 624 (34.8%)
epoch 7 test : #Correct 209 on 624 (33.5%)
epoch 8 test : #Correct 231 on 624 (37.0%)
epoch 9 test : #Correct 242 on 624 (38.8%)
epoch 10 test : #Correct 241 on 624 (38.6%)
epoch 11 test : #Correct 242 on 624 (38.8%)
epoch 12 test : #Correct 225 on 624 (36.1%)
epoch 13 test : #Correct 240 on 624 (38.5%)
epoch 14 test : #Correct 243 on 624 (38.9%)
epoch 15 test : #Correct 253 on 624 (40.5%)
epoch 16 test : #Correct 225 on 624 (36.1%)
epoch 17 test : #Correct 234 on 624 (37.5%)
epoch 18 test : #Correct 258 on 624 (41.3%)
epoch 19 test : #Correct 256 on 624 (41.0%)
epoch 20 test : #Correct 242 on 624 (38.8%)
epoch 21 test : #Correct 228 on 624 (36.5%)
epoch 22 test : #Correct 260 on 624 (41.7%

In [12]:
torch.save(alexTunedClassifier, "alexFineTuned-50epoch-lr-0.01.ckpt")

In [ ]:
optimizer=optim.SGD([{'params': alexTunedClassifier.classifier.parameters()},
                     {'params': alexTunedClassifier.features.parameters(), 'lr': 0.0}
                    ], lr=0.001, momentum=0.9)
for epoch in range(50): 
    print "epoch", epoch, 
    trainclassifier(alexTunedClassifier, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClass, 1)
    testClassifier(alexTunedClassifier, dImgTest, imSize, imageTestTransform, ImgNameToIdClass, 1)

 epoch 0 test : #Correct 281 on 624 (45.0%)
epoch 1 test : #Correct 284 on 624 (45.5%)
epoch 2 test : #Correct 289 on 624 (46.3%)
epoch 3 test : #Correct 290 on 624 (46.5%)
epoch 4 test : #Correct 290 on 624 (46.5%)
epoch 5 test : #Correct 287 on 624 (46.0%)
epoch 6 test : #Correct 288 on 624 (46.2%)
epoch 7 test : #Correct 286 on 624 (45.8%)
epoch 8 test : #Correct 289 on 624 (46.3%)
epoch 9 test : #Correct 288 on 624 (46.2%)
epoch 10 test : #Correct 289 on 624 (46.3%)
epoch 11 test : #Correct 291 on 624 (46.6%)
epoch 12 test : #Correct 291 on 624 (46.6%)
epoch 13 test : #Correct 288 on 624 (46.2%)
epoch 15 test : #Correct 289 on 624 (46.3%)
epoch 16 test : #Correct 287 on 624 (46.0%)
epoch 17 test : #Correct 293 on 624 (47.0%)
epoch 18 test : #Correct 291 on 624 (46.6%)
epoch 19 test : #Correct 295 on 624 (47.3%)
epoch 20 test : #Correct 292 on 624 (46.8%)
epoch 21 test : #Correct 293 on 624 (47.0%)
epoch 22 test : #Correct 291 on 624 (46.6%)
epoch 23 test : #Correct 293 on 624 (47.0

In [ ]:
torch.save(alexTunedClassifier, "alexFineTuned-50epoch-lr-0.01-50epoch-lr-0.001.ckpt")

In [ ]:
alexTunedClassifier = torch.load("alexFineTuned-50epoch-lr-0.01-50epoch-lr-0.001.ckpt")
alexTunedClassifier = alexTunedClassifier.cuda()

optimizer=optim.SGD(alexTunedClassifier.parameters(), lr=0.01, momentum=0.9)
for epoch in range(50): # loop over the dataset multiple times
    print "epoch", epoch, 
    trainclassifier(alexTunedClassifier, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClass, 1)
    testClassifier(alexTunedClassifier, dImgTest, imSize, imageTestTransform, ImgNameToIdClass, 1)

In [ ]:
torch.save(alexTunedClassifier, "alexFineTuned-50epoch-lr-0.01-50epoch-lr-0.001-50epoch-lr-0.01.ckpt")

In [ ]:
optimizer=optim.SGD(alexTunedClassifier.parameters(), lr=0.001, momentum=0.9)
for epoch in range(50): 
    print "epoch", epoch, 
    trainclassifier(alexTunedClassifier, optimizer, criterion, batchSize, dImgTrain, imSize, imageTrainTransform, ImgNameToIdClass, 1)
    testClassifier(alexTunedClassifier, dImgTest, imSize, imageTestTransform, ImgNameToIdClass, 1)

In [ ]:
torch.save(alexTunedClassifier, "alexFineTuned-50epoch-lr-0.01-50epoch-lr-0.001-50epoch-lr-0.01-50epoch-lr-0.001.ckpt")